In [1]:
import pandas as pd
from glob import  glob
import json
from tqdm.notebook import trange, tqdm
from sklearn.model_selection import StratifiedKFold
import numpy as np

In [2]:
label_path = glob('./train/annotations/*')

In [3]:
label_path

['./train/annotations/003e852352bb.json',
 './train/annotations/0000ae6cbdb1.json',
 './train/annotations/0042a14962ca.json',
 './train/annotations/0003a50817cf.json',
 './train/annotations/003f04b79fa1.json',
 './train/annotations/0005413054c9.json',
 './train/annotations/0045659a08df.json',
 './train/annotations/0005e64fdc6e.json',
 './train/annotations/0045b0778040.json',
 './train/annotations/000614d3eab6.json',
 './train/annotations/0045fa6e26a7.json',
 './train/annotations/000631f33080.json',
 './train/annotations/004f0e75b1d5.json',
 './train/annotations/000917f5d829.json',
 './train/annotations/004f1730f1b7.json',
 './train/annotations/000944919c5c.json',
 './train/annotations/00511c0df1ee.json',
 './train/annotations/000a2d157ea1.json',
 './train/annotations/00534e0db796.json',
 './train/annotations/000b92c3b098.json',
 './train/annotations/005559e9d574.json',
 './train/annotations/000cd10b6cda.json',
 './train/annotations/0055bcadaffd.json',
 './train/annotations/000d269c8e26

In [4]:
len(label_path)

60578

In [5]:
image_path = glob('./train/images/*')

In [6]:
len(image_path)

60578

In [7]:
image_path = []
source = []
gts = []
types = []
for i in  tqdm(label_path):
    with open(i) as f:
        data = json.loads(f.read())
    image_path.append(i.replace('annotations', 'images').replace('json', 'jpg'))
    source.append(data['source'])
    gts.append(data['data-series'])
    types.append(data['chart-type'])

  0%|          | 0/60578 [00:00<?, ?it/s]

In [8]:
df = pd.DataFrame({'image_path':image_path,
                   'source':source,
                   'gts':gts,
                   'types':types
                   })

In [9]:
df

,image_path,source,gts,types
0,./train/images/003e852352bb.jpg,generated,"[{'x': 'Afghanistan', 'y': 230803570.432223}, ...",line
1,./train/images/0000ae6cbdb1.jpg,generated,"[{'x': 'Malawi', 'y': 837799.95244483}, {'x': ...",vertical_bar
2,./train/images/0042a14962ca.jpg,generated,"[{'x': 'Grant', 'y': 85.85953569653202}, {'x':...",vertical_bar
3,./train/images/0003a50817cf.jpg,generated,"[{'x': 'Jan', 'y': 34.06176711672003}, {'x': '...",line
4,./train/images/003f04b79fa1.jpg,generated,"[{'x': 'Vanuatu', 'y': 77.05247474888785}, {'x...",line
...,...,...,...,...
60573,./train/images/fffc786ec1bf.jpg,generated,"[{'x': 'Iceland', 'y': 106.50554321027937}, {'...",vertical_bar
60574,./train/images/fffdafef9db0.jpg,generated,"[{'x': '2005', 'y': 122.59042313521488}, {'x':...",line
60575,./train/images/fffdb0a69d5f.jpg,generated,"[{'x': 'Denmark', 'y': 47.10673686341258}, {'x...",line
60576,./train/images/fffdc83973fc.jpg,generated,"[{'x': '40', 'y': 7}, {'x': '50', 'y': 7}, {'x...",dot


In [10]:
def preprocess_gts(input):
    output = '<x_start>'
    xs = []
    ys = []
    for i in input:
        xs.append(str(i['x']))
        ys.append(str(i['y']))
    output += ';'.join(xs) + '<x_end> <y_start>' + ';'.join(ys) + '<y_end>'
    return output
df['label'] = df['gts'].apply(lambda x:preprocess_gts(x))


In [14]:
df['gts'].values[1]

[{'x': 'Malawi', 'y': 837799.95244483},
 {'x': 'Malaysia', 'y': 960480.6137135166},
 {'x': 'Maldives', 'y': 750170.9086814824},
 {'x': 'Mali', 'y': 697593.4824234738},
 {'x': 'Malta', 'y': 574912.8211547872},
 {'x': 'Martinique', 'y': 452232.1598861007},
 {'x': 'Mauritania', 'y': 653778.9605418},
 {'x': 'Mauritius', 'y': 539861.2036494482},
 {'x': 'Mayotte', 'y': 417180.54238076165},
 {'x': 'Melanesia', 'y': 574912.8211547872}]

In [15]:
df['label'].values[1]

'<x_start>Malawi;Malaysia;Maldives;Mali;Malta;Martinique;Mauritania;Mauritius;Mayotte;Melanesia<x_end> <y_start>837799.95244483;960480.6137135166;750170.9086814824;697593.4824234738;574912.8211547872;452232.1598861007;653778.9605418;539861.2036494482;417180.54238076165;574912.8211547872<y_end>'

In [16]:
df

,image_path,source,gts,types,label
0,./train/images/003e852352bb.jpg,generated,"[{'x': 'Afghanistan', 'y': 230803570.432223}, ...",line,<x_start>Afghanistan;Africa;Albania;Algeria;An...
1,./train/images/0000ae6cbdb1.jpg,generated,"[{'x': 'Malawi', 'y': 837799.95244483}, {'x': ...",vertical_bar,<x_start>Malawi;Malaysia;Maldives;Mali;Malta;M...
2,./train/images/0042a14962ca.jpg,generated,"[{'x': 'Grant', 'y': 85.85953569653202}, {'x':...",vertical_bar,<x_start>Grant;Greene;Hamilton;Hancock;Harriso...
3,./train/images/0003a50817cf.jpg,generated,"[{'x': 'Jan', 'y': 34.06176711672003}, {'x': '...",line,<x_start>Jan;Feb;Mar;Apr;May;Jun;Jul;Aug;Sep;O...
4,./train/images/003f04b79fa1.jpg,generated,"[{'x': 'Vanuatu', 'y': 77.05247474888785}, {'x...",line,<x_start>Vanuatu;Vatican;Venezuela;Vietnam;Wal...
...,...,...,...,...,...
60573,./train/images/fffc786ec1bf.jpg,generated,"[{'x': 'Iceland', 'y': 106.50554321027937}, {'...",vertical_bar,<x_start>Iceland;India;Indonesia;Iran;Iraq;Ire...
60574,./train/images/fffdafef9db0.jpg,generated,"[{'x': '2005', 'y': 122.59042313521488}, {'x':...",line,<x_start>2005;2006;2007;2008;2009;2010;2011;20...
60575,./train/images/fffdb0a69d5f.jpg,generated,"[{'x': 'Denmark', 'y': 47.10673686341258}, {'x...",line,<x_start>Denmark;Djibouti;Dominican Republic;E...
60576,./train/images/fffdc83973fc.jpg,generated,"[{'x': '40', 'y': 7}, {'x': '50', 'y': 7}, {'x...",dot,<x_start>40;50;60;70;80;90+<x_end> <y_start>7;...


In [17]:
sfk =  StratifiedKFold(5)
df['fold'] = -1
for idx, (_, test) in enumerate(sfk.split(df, df['types'])):
    df.loc[test, 'fold'] = idx

In [18]:
df

,image_path,source,gts,types,label,fold
0,./train/images/003e852352bb.jpg,generated,"[{'x': 'Afghanistan', 'y': 230803570.432223}, ...",line,<x_start>Afghanistan;Africa;Albania;Algeria;An...,0
1,./train/images/0000ae6cbdb1.jpg,generated,"[{'x': 'Malawi', 'y': 837799.95244483}, {'x': ...",vertical_bar,<x_start>Malawi;Malaysia;Maldives;Mali;Malta;M...,0
2,./train/images/0042a14962ca.jpg,generated,"[{'x': 'Grant', 'y': 85.85953569653202}, {'x':...",vertical_bar,<x_start>Grant;Greene;Hamilton;Hancock;Harriso...,0
3,./train/images/0003a50817cf.jpg,generated,"[{'x': 'Jan', 'y': 34.06176711672003}, {'x': '...",line,<x_start>Jan;Feb;Mar;Apr;May;Jun;Jul;Aug;Sep;O...,0
4,./train/images/003f04b79fa1.jpg,generated,"[{'x': 'Vanuatu', 'y': 77.05247474888785}, {'x...",line,<x_start>Vanuatu;Vatican;Venezuela;Vietnam;Wal...,0
...,...,...,...,...,...,...
60573,./train/images/fffc786ec1bf.jpg,generated,"[{'x': 'Iceland', 'y': 106.50554321027937}, {'...",vertical_bar,<x_start>Iceland;India;Indonesia;Iran;Iraq;Ire...,4
60574,./train/images/fffdafef9db0.jpg,generated,"[{'x': '2005', 'y': 122.59042313521488}, {'x':...",line,<x_start>2005;2006;2007;2008;2009;2010;2011;20...,4
60575,./train/images/fffdb0a69d5f.jpg,generated,"[{'x': 'Denmark', 'y': 47.10673686341258}, {'x...",line,<x_start>Denmark;Djibouti;Dominican Republic;E...,4
60576,./train/images/fffdc83973fc.jpg,generated,"[{'x': '40', 'y': 7}, {'x': '50', 'y': 7}, {'x...",dot,<x_start>40;50;60;70;80;90+<x_end> <y_start>7;...,4


In [19]:
df.groupby(['fold', 'types'])['types'].count()

fold  types         
0     dot               1026
      horizontal_bar      15
      line              4989
      scatter           2248
      vertical_bar      3838
1     dot               1026
      horizontal_bar      15
      line              4989
      scatter           2249
      vertical_bar      3837
2     dot               1026
      horizontal_bar      15
      line              4988
      scatter           2249
      vertical_bar      3838
3     dot               1026
      horizontal_bar      14
      line              4988
      scatter           2249
      vertical_bar      3838
4     dot               1027
      horizontal_bar      14
      line              4988
      scatter           2248
      vertical_bar      3838
Name: types, dtype: int64

In [18]:
df.to_csv('train_with_fold.csv', index=None)